In [ ]:
import ee
ee.Initialize()

In [ ]:
# Import Libraries
import geemap
from pprint import pprint
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rxr
import shapely as shp
import os
import re

In [ ]:
# Get Sentinel-2 Mosaic to Align Planet Data
MAX_CLOUD_PROBABILITY = 50
geometry = ee.Geometry({
    'type': 'Polygon',
    'coordinates': [[
        [66, 66],
        [66, 73.5],
        [85, 73.5],
        [85, 66],
        [66, 66]
        ]]
})

s2 = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(geometry)
    .filterDate('2017-06-15', '2021-08-31')
    .filter(ee.Filter.dayOfYear(135, 243))
);

s2_clouds = (
    ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
    .filterBounds(geometry)
    .filterDate('2017-06-15', '2021-08-31')
    .filter(ee.Filter.dayOfYear(135, 243))
);

def s2_maskcloud(image):
    
    clouds = ee.Image(image.get('cloud_mask')).select('probability')
    
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    
    return image.updateMask(isNotCloud)

# Join S2 SR with cloud probability dataset to add cloud mask.
s2_with_clouds = ee.Join.saveFirst('cloud_mask').apply(    
  primary=s2,
  secondary=s2_clouds,
  condition=ee.Filter.equals(leftField='system:index', rightField='system:index'))

s2_masked = ee.ImageCollection(s2_with_clouds).map(s2_maskcloud)

s2_mosaic = (
    s2_masked
    .select(['B2', 'B3', 'B4', 'B8'])
    .median()
    .clip(geometry));

pprint(s2_mosaic.getInfo())

In [ ]:
Map = geemap.Map()
Map.centerObject(geometry)
Map.addLayer(geometry, {'color': 'black'}, 'AOI')
Map.addLayer(s2_mosaic, {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':1000}, 'S2 All Years')

In [ ]:
Map

In [ ]:
# Export only in the polygons we need
# Import shapefile with AOI (multipolygon)
aoi = gpd.read_file("/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/yg_train_regions/rts_buffer/RTS_buffer.shp")
# convert from multipolygon to multiple polygons
aoi = aoi.explode(column = 'geometry', ignore_index = True)
# remove inner holes
aoi.geometry = aoi.geometry.exterior
pprint(aoi.geometry)
# convert back to polygon
aoi.geometry = [shp.geometry.Polygon([shp.geometry.Point(x, y) for x, y in list(feature.coords)]) for feature in aoi.geometry]
aoi.geometry = aoi.geometry.to_crs(crs = 32642).buffer(100, join_style = 2).to_crs(crs = 4326)
pprint(aoi.geometry)

In [ ]:
# figure out the correct utm zone for each polygon
base_dir = '/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/planet_data/yg_train_regions'
tgt_files = []
for root, subdirs, files in os.walk(base_dir):
    for file in files:
        if re.match('.*SR.*tif$', file):
            tgt_files.append(os.path.join(root, file))
            
tgt_files = sorted(tgt_files)
tgt_files = [file for file in tgt_files if not re.match('.*/2017/.*tif$', file)]

utm42n = []
utm43n = []
utm44n = []
for polygon_id in range(0, 68):
    example_files = [file for file in tgt_files if re.match('.*/polygon_id_' + str(polygon_id) + '/.*', file)]
    
    crs = []
    for file in example_files:
        crs.append(str(rxr.open_rasterio(file).rio.crs).split(':')[1][3:5])
    
    crs = list(set(crs))
    
    if '42' in crs:
        utm42n.append(polygon_id)
    
    if '43' in crs:
        utm43n.append(polygon_id)
    
    if '44' in crs:
        utm44n.append(polygon_id)
    
    print('\n')

print(utm42n)
print(utm43n)
print(utm44n)

In [ ]:
aoi_utm42n = aoi.iloc[utm42n]
for idx, row in enumerate(aoi_utm42n.iterrows()):
    polygon_id = row[0]
    description = 'Sentinel-2_YG_Mosaic_bgrnir_UTM42N_' + str(polygon_id)
    name = 'planet_processing/data/yg_train_regions/sentinel_data/' + description
    img_geometry = [[[x, y] for x, y in list(aoi.geometry[polygon_id].exterior.coords)]]
    
    # Export 2017-2021 growing season Sentinel-2 Mosaic to Drive
    task = ee.batch.Export.image.toCloudStorage(
        image = s2_mosaic,
        description = description,
        bucket = 'abrupt_thaw',
        fileNamePrefix = name,
        crs = 'EPSG:32642',
        scale = 10,
        maxPixels = 1e13,
        region = img_geometry,
        fileFormat = 'GeoTIFF',
        formatOptions = {
            'cloudOptimized': True
        }
    )
    task.start()
    

In [ ]:
aoi_utm43n = aoi.iloc[utm43n]
for idx, row in enumerate(aoi_utm43n.iterrows()):
    polygon_id = row[0]
    description = 'Sentinel-2_YG_Mosaic_bgrnir_UTM43N_' + str(polygon_id)
    name = 'planet_processing/data/yg_train_regions/sentinel_data/' + description
    img_geometry = [[[x, y] for x, y in list(aoi.geometry[polygon_id].exterior.coords)]]
    
    # Export 2017-2021 growing season Sentinel-2 Mosaic to Drive
    task = ee.batch.Export.image.toCloudStorage(
        image = s2_mosaic,
        description = description,
        bucket = 'abrupt_thaw',
        fileNamePrefix = name,
        crs = 'EPSG:32643',
        scale = 10,
        maxPixels = 1e13,
        region = img_geometry,
        fileFormat = 'GeoTIFF',
        formatOptions = {
            'cloudOptimized': True
        }
    )
    task.start()

In [ ]:
aoi_utm44n = aoi.iloc[utm44n]
for idx, row in enumerate(aoi_utm44n.iterrows()):
    polygon_id = row[0]
    description = 'Sentinel-2_YG_Mosaic_bgrnir_UTM44N_' + str(polygon_id)
    name = 'planet_processing/data/yg_train_regions/sentinel_data/' + description
    img_geometry = [[[x, y] for x, y in list(aoi.geometry[polygon_id].exterior.coords)]]
    
    # Export 2017-2021 growing season Sentinel-2 Mosaic to Drive
    task = ee.batch.Export.image.toCloudStorage(
        image = s2_mosaic,
        description = description,
        bucket = 'abrupt_thaw',
        fileNamePrefix = name,
        crs = 'EPSG:32644',
        scale = 10,
        maxPixels = 1e13,
        region = img_geometry,
        fileFormat = 'GeoTIFF',
        formatOptions = {
            'cloudOptimized': True
        }
    )
    task.start()